In [ ]:
import IPython.display as display
!pip install --no-index --no-deps /kaggle/input/mmdetectron-31-wheel/pycocotools-2.0.6-cp310-cp310-linux_x86_64.whl
!pip install --no-index --no-deps /kaggle/input/mmdetectron-31-wheel/torch-1.12.1+cu116-cp310-cp310-linux_x86_64.whl
!pip install --no-index --no-deps /kaggle/input/mmdetectron-31-wheel/torchvision-0.13.1+cu116-cp310-cp310-linux_x86_64.whl
!pip install --no-index --no-deps /kaggle/input/mmdetectron-31-wheel/mmcv-2.0.1-cp310-cp310-manylinux1_x86_64.whl 
!pip install --no-index --no-deps /kaggle/input/mmdetectron-31-wheel/openmim-0.3.9-py2.py3-none-any.whl
!pip install --no-index --no-deps /kaggle/input/mmdetectron-31-wheel/mmengine-0.7.4-py3-none-any.whl
!pip install --no-index --no-deps /kaggle/input/mmdetectron-31-wheel/addict-2.4.0-py3-none-any.whl
!pip install yapf==0.40.1
display.clear_output()

In [ ]:
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection
!pip install -v -e .

!mkdir checkpoint
!mkdir outputs
!mkdir data
!mkdir data/images
!mkdir data/labels

display.clear_output()

In [ ]:
import torch, torchvision,mmdet
print("torch=",torch.__version__,torch.cuda.is_available())
print("mmdet=",mmdet.__version__)
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

In [ ]:
import os
import io
import shutil
from PIL import Image

In [ ]:
from tqdm.notebook import tqdm
import pandas as pd

import matplotlib.pyplot as plt
from PIL import Image

from glob import glob
import matplotlib.pyplot as plt

In [ ]:
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

import mmdet as mmdet
print(mmdet.__version__)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import GroupKFold
from pathlib import Path
from pycocotools.coco import COCO
from PIL import Image
import cv2
import os

In [ ]:
test_dir = '/kaggle/input/datasets/kagglertw/dal-shemagh/dal-shemagh-detection-challenge/images/test'
from tqdm.notebook import tqdm
from pycocotools import mask as maskUtils
from joblib import Parallel, delayed
import json, itertools

In [ ]:
from mmdet.apis import inference_detector
from mmdet.apis import init_detector

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [ ]:
mmdet_config = [
    
    {
        "config": f"/kaggle/input/notebooks/kagglertw/shemagh-simple-ddq-map-50-o3/mmdetection/configs/ddq/ddq-detr-4scale_swinl_1xb1-15e_sky.py", 
        "weight": f"/kaggle/input/notebooks/kagglertw/shemagh-simple-ddq-map-50-o3/mmdetection/work_dirs/ddq-detr-4scale_swinl_1xb1-15e_sky/best_coco_bbox_mAP_epoch_42.pth"
    }, 

]

In [ ]:
mmdet_model_list = []
for mmdet_dic in mmdet_config:
    cfg = mmdet_dic["config"]
    weight = mmdet_dic["weight"]  
    print(cfg)
    print(f"Loading weight: {weight}")
    
    
    model = init_detector(cfg, weight, device)
    mmdet_model_list.append(model)

In [ ]:
result_mmdet_path = f'/kaggle/tmp/mmdet_result/'
os.makedirs(result_mmdet_path, exist_ok=True)

In [ ]:
import random
import torch
import os
import numpy as np

def seed_torch(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    
    torch.backends.cudnn.deterministic = True

seed_torch(seed=42)

In [ ]:
import pandas as pd
import glob
import os


test = pd.DataFrame(columns=["ID"])


image_files = glob.glob("/kaggle/input/datasets/kagglertw/dal-shemagh/dal-shemagh-detection-challenge/images/test/*.jpg")


test["ID"] = [os.path.splitext(os.path.basename(f))[0] for f in image_files]


print(test.head())

In [ ]:
test_df = test.copy()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import pandas as pd
import pickle
from time import time
from tqdm import tqdm
from PIL import Image
import torch
import mmcv
from torchvision.ops import nms  




iou_thr = 0.4
n_show = 20


conf_score_class_0 = 0.3  
conf_score_class_1 = 0.15 

device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device)


model_predictions_list = []
fused_predictions_list = []
time_mmdet_infer = []

draw = True


for i in tqdm(range(len(test_df))):
    row = test_df.iloc[i]
    image_id = row["ID"]

    img_path = f'/kaggle/input/datasets/kagglertw/dal-shemagh/dal-shemagh-detection-challenge/images/test/{image_id}.jpg'
    img = mmcv.imread(img_path)

    height, width = img.shape[:2]  

    
    boxes_list = []
    scores_list = []
    labels_list = []

    st = time()

    
    model = mmdet_model_list[0]
    
    
    det_result = inference_detector(model, img)
    pred_instances = det_result.pred_instances  

    
    bboxes = pred_instances.bboxes  
    scores = pred_instances.scores  
    labels = pred_instances.labels  

    
    
    class_0_mask = (labels == 0) & (scores >= conf_score_class_0)
    class_1_mask = (labels == 1) & (scores >= conf_score_class_1)
    
    
    high_conf_indices = class_0_mask | class_1_mask
    
    bboxes = bboxes[high_conf_indices]
    scores = scores[high_conf_indices]
    labels = labels[high_conf_indices]

    
    keep_indices = nms(bboxes, scores, iou_thr)
    bboxes = bboxes[keep_indices].cpu().numpy()  
    scores = scores[keep_indices].cpu().numpy()  
    labels = labels[keep_indices].cpu().numpy()  

    
    bboxes_normalized = bboxes.copy()
    bboxes_normalized[:, [0, 2]] /= width   
    bboxes_normalized[:, [1, 3]] /= height  

    
    boxes_list.append(bboxes_normalized.tolist())  
    scores_list.append(scores.tolist())
    labels_list.append(labels.tolist())
    
    
    if len(scores) > 0:  
        model_predictions = np.concatenate([ 
            np.array([[image_id]] * len(scores)),  
            labels.reshape(-1, 1),                 
            scores.reshape(-1, 1),                 
            bboxes                                  
        ], axis=1)

        model_df = pd.DataFrame(model_predictions, columns=['Image_ID', 'class', 'confidence', 'xmin', 'ymin', 'xmax', 'ymax'])
        model_predictions_list.append(model_df)

    time_mmdet_infer.append(time() - st)

    
    if draw and i < n_show: 
        fig, ax = plt.subplots(1, figsize=(12, 8))
        ax.imshow(img)

        
        if len(model_predictions_list) > 0:
            for bbox in model_predictions_list[-1].loc[model_predictions_list[-1]['Image_ID'] == image_id].values:
                xmin, ymin, xmax, ymax = map(float, bbox[3:7])  
                rect = patches.Rectangle(
                    (xmin, ymin), xmax - xmin, ymax - ymin,
                    linewidth=2, edgecolor='r', facecolor='none'
                )
                ax.add_patch(rect)

        plt.title(f"Model Detection for {image_id}")
        plt.show()
            

if model_predictions_list:
    model_1_df = pd.concat(model_predictions_list, ignore_index=True)  
else:
    model_1_df = pd.DataFrame(columns=['Image_ID', 'class', 'confidence', 'xmin', 'ymin', 'xmax', 'ymax'])


print("Model 1 Predictions:")
print(model_1_df.head())

In [ ]:
model_1_df

In [ ]:
model_1_df['Image_ID'].nunique()

In [ ]:
def count_png_files(folder_path):
    count = 0
    for file in os.listdir(folder_path):
        if file.lower().endswith(".jpg"):
            count += 1
    return count

In [ ]:
count_png_files("/kaggle/input/datasets/kagglertw/dal-shemagh/dal-shemagh-detection-challenge/images/test")

In [ ]:
import glob
import os


path = "/kaggle/input/datasets/kagglertw/dal-shemagh/dal-shemagh-detection-challenge/images/test/*.jpg"


ids = [os.path.splitext(os.path.basename(f))[0] for f in glob.glob(path)]


print(ids[:10])

In [ ]:
rest = list(set(ids) - set(model_1_df['Image_ID'].unique()))

In [ ]:
import matplotlib.pyplot as plt
import random
from PIL import Image
import os


sample_ids = random.sample(rest, 3)


folder = "/kaggle/input/datasets/kagglertw/dal-shemagh/dal-shemagh-detection-challenge/images/test"


plt.figure(figsize=(12, 4))
for i, img_id in enumerate(sample_ids):
    img_path = os.path.join(folder, f"{img_id}.jpg")
    img = Image.open(img_path)
    
    plt.subplot(1, 3, i+1)
    plt.imshow(img)
    plt.axis('off')
    plt.title(img_id)

plt.show()

In [ ]:
result_df = model_1_df.copy(deep = True)

In [ ]:
result_df.shape

In [ ]:
len(rest)

In [ ]:
for i in rest:
    result_df = pd.concat([result_df, pd.DataFrame([{
        "Image_ID": i,
        "class": "-",
        "confidence": "-",
        "xmin": "-",
        "ymin": "-",
        "xmax": "-",
        "ymax": "-"
    }])], ignore_index=True)

In [ ]:
result_df.shape

In [ ]:
sub = result_df.copy()

In [ ]:
import pandas as pd

df = result_df.copy()


df["class"] = pd.to_numeric(df["class"], errors="coerce")

def compute_right_place(group):
    has_class_0 = (group["class"] == 0).any()
    has_class_1 = (group["class"] == 1).any()
    
    if has_class_0 and has_class_1:
        return 1
    else:
        return 0


df["right_place"] = (
    df.groupby("Image_ID")
      .apply(compute_right_place)
      .reindex(df["Image_ID"])
      .values
)

In [ ]:
result_df = df.copy(deep = True)

In [ ]:
rr = result_df[result_df['right_place'] == 1]

In [ ]:
rr_id = rr['Image_ID'].unique()

In [ ]:
result_df

In [ ]:
%%capture
import pandas as pd
import numpy as np


result_df['xmin'] = pd.to_numeric(result_df['xmin'], errors='coerce')
result_df['ymin'] = pd.to_numeric(result_df['ymin'], errors='coerce')
result_df['xmax'] = pd.to_numeric(result_df['xmax'], errors='coerce')
result_df['ymax'] = pd.to_numeric(result_df['ymax'], errors='coerce')


def get_box_center(row):
    x_center = (row['xmin'] + row['xmax']) / 2
    y_center = (row['ymin'] + row['ymax']) / 2
    return x_center, y_center


def calculate_distance(center1, center2):
    return np.sqrt((center1[0] - center2[0])**2 + (center1[1] - center2[1])**2)


def calculate_iou(box1, box2):
    
    
    
    x_left = max(box1['xmin'], box2['xmin'])
    y_top = max(box1['ymin'], box2['ymin'])
    x_right = min(box1['xmax'], box2['xmax'])
    y_bottom = min(box1['ymax'], box2['ymax'])
    
    
    if x_right < x_left or y_bottom < y_top:
        return 0.0
    
    
    intersection_area = (x_right - x_left) * (y_bottom - y_top)
    
    
    box1_area = (box1['xmax'] - box1['xmin']) * (box1['ymax'] - box1['ymin'])
    box2_area = (box2['xmax'] - box2['xmin']) * (box2['ymax'] - box2['ymin'])
    
    
    union_area = box1_area + box2_area - intersection_area
    
    
    iou = intersection_area / union_area if union_area > 0 else 0.0
    
    return iou


image_ids_with_both = []
grouped = result_df.groupby('Image_ID')

for image_id, group in grouped:
    
    valid_group = group[group['class'].notna()]
    classes = valid_group['class'].unique()
    
    
    if 0.0 in classes and 1.0 in classes:
        image_ids_with_both.append(image_id)

print(f"Image IDs with both classes: {image_ids_with_both}")


result_dict = {}

for image_id in image_ids_with_both:
    group = result_df[result_df['Image_ID'] == image_id]
    
    
    class_1_boxes = group[(group['class'] == 1.0) & 
                          (group['xmin'].notna()) & 
                          (group['xmax'].notna())].copy()
    
    class_0_boxes = group[(group['class'] == 0.0) & 
                          (group['xmin'].notna()) & 
                          (group['xmax'].notna())].copy()
    
    if len(class_1_boxes) == 0 or len(class_0_boxes) == 0:
        continue
    
    
    class_1_row = class_1_boxes.iloc[0]
    class_1_center = get_box_center(class_1_row)
    
    
    min_distance = float('inf')
    closest_class_0_row = None
    
    for idx, class_0_row in class_0_boxes.iterrows():
        class_0_center = get_box_center(class_0_row)
        distance = calculate_distance(class_1_center, class_0_center)
        
        if distance < min_distance:
            min_distance = distance
            closest_class_0_row = class_0_row
    
    
    result_dict[image_id] = {
        'class_1_coords': {
            'xmin': class_1_row['xmin'],
            'ymin': class_1_row['ymin'],
            'xmax': class_1_row['xmax'],
            'ymax': class_1_row['ymax']
        },
        'class_0_coords': {
            'xmin': closest_class_0_row['xmin'],
            'ymin': closest_class_0_row['ymin'],
            'xmax': closest_class_0_row['xmax'],
            'ymax': closest_class_0_row['ymax']
        },
        'class_1_center': class_1_center,
        'class_0_center': get_box_center(closest_class_0_row)
    }

print(f"\nResult dictionary: {result_dict}")


summary_data = []

for image_id, data in result_dict.items():
    class_1_center_y = data['class_1_center'][1]
    class_0_center_y = data['class_0_center'][1]
    
    
    class_1_upper = data['class_1_coords']['ymin']
    class_0_upper = data['class_0_coords']['ymin']
    
    
    iou = calculate_iou(data['class_1_coords'], data['class_0_coords'])
    
    
    
    is_above = class_0_center_y < class_1_center_y
    
    
    
    if class_1_upper < class_0_upper and iou > 0.005:
        is_above = False
    
    summary_data.append({
        'Image_ID': image_id,
        'class_1_xmin': data['class_1_coords']['xmin'],
        'class_1_ymin': data['class_1_coords']['ymin'],
        'class_1_xmax': data['class_1_coords']['xmax'],
        'class_1_ymax': data['class_1_coords']['ymax'],
        'class_0_xmin': data['class_0_coords']['xmin'],
        'class_0_ymin': data['class_0_coords']['ymin'],
        'class_0_xmax': data['class_0_coords']['xmax'],
        'class_0_ymax': data['class_0_coords']['ymax'],
        'iou': iou,
        'above': is_above
    })

summary_df = pd.DataFrame(summary_data)
print("\nSummary DataFrame:")
print(summary_df)

In [ ]:
summary_df['above'].value_counts()

In [ ]:
filterred = summary_df[summary_df['above'] == True]

In [ ]:
filterred_ids = filterred['Image_ID'].unique()

In [ ]:
image_folder = "/kaggle/input/datasets/kagglertw/dal-shemagh/dal-shemagh-detection-challenge/images/test"


numeric_cols = ["confidence", "xmin", "ymin", "xmax", "ymax"]
for col in numeric_cols:
    result_df[col] = pd.to_numeric(result_df[col], errors='coerce')  


def convert_bbox(row, width, height):
    if pd.isna(row['confidence']) or pd.isna(row['xmin']) or pd.isna(row['ymin']):
        return None  
    cx = (row['xmin'] + row['xmax']) / 2 / width
    cy = (row['ymin'] + row['ymax']) / 2 / height
    w = (row['xmax'] - row['xmin']) / width
    h = (row['ymax'] - row['ymin']) / height
    return f"{int(row['class'])} {row['confidence']:.6f} {cx:.6f} {cy:.6f} {w:.6f} {h:.6f}"

submission_list = []


for img_id, group in result_df.groupby("Image_ID"):
    img_path = os.path.join(image_folder, f"{img_id}.jpg")
    if os.path.exists(img_path):
        with Image.open(img_path) as img:
            width, height = img.size
    else:
        width, height = 640, 640  

    
    pred_strings = [convert_bbox(r, width, height) for _, r in group.iterrows()]
    
    pred_strings = [p for p in pred_strings if p is not None]

    if len(pred_strings) == 0:
        prediction_string = "-"
    else:
        prediction_string = " ".join(pred_strings)

    right_place_value = group["right_place"].iloc[0]
    
    submission_list.append({
        "filename": f"{img_id}.jpg",
        "right_place": right_place_value,
        "prediction_string": prediction_string
    })


submission_df = pd.DataFrame(submission_list)


submission_df.to_csv("/kaggle/working/submission.csv", index=False)

print(submission_df.head())

In [ ]:

for idx, row in submission_df.iterrows():
    if row['prediction_string'] == "-":
        submission_df.at[idx, 'right_place'] = 0  



In [ ]:
for idd in filterred_ids:
    submission_df.loc[
        submission_df['filename'] == f"{idd}.jpg",
        'right_place'
    ] = 0

In [ ]:
submission_df.to_csv("/kaggle/working/submission.csv", index=False)

In [ ]:
submission_df.head()